In [21]:
import gym

env = gym.make("MountainCar-v0")
env.reset()

print(env.observation_space.high)
print(env.observation_space.low)
print(env.action_space)
done = False

while not done:
    action = 2
    new_state, reward , done, _ = env.step(action)
    print(new_state)
    env.render()

env.close()

[0.6  0.07]
[-1.2  -0.07]
Discrete(3)
[-0.57579491  0.00140024]
[-0.57300479  0.00279012]
[-0.56884548  0.00415931]
[-0.56334787  0.00549762]
[-0.55655284  0.00679503]
[-0.54851106  0.00804178]
[-0.53928261  0.00922845]
[-0.52893658  0.01034603]
[-0.51755051  0.01138607]
[-0.50520981  0.01234071]
[-0.49200695  0.01320286]
[-0.47804067  0.01396627]
[-0.46341502  0.01462565]
[-0.4482383   0.01517672]
[-0.43262198  0.01561632]
[-0.41667954  0.01594243]
[-0.40052532  0.01615422]
[-0.38427332  0.016252  ]
[-0.3680361   0.01623722]
[-0.35192371  0.01611239]
[-0.33604271  0.015881  ]
[-0.32049532  0.01554739]
[-0.30537869  0.01511663]
[-0.29078428  0.01459441]
[-0.27679744  0.01398684]
[-0.26349706  0.01330038]
[-0.25095542  0.01254164]
[-0.23923812  0.01171731]
[-0.22840409  0.01083403]
[-0.21850577  0.00989832]
[-0.20958929  0.00891649]
[-0.20169468  0.0078946 ]
[-0.19485622  0.00683847]
[-0.18910262  0.00575359]
[-0.18445741  0.00464522]
[-0.18093909  0.00351832]
[-0.17856141  0.00237768]
